In [1]:
!pip install --upgrade datasets

In [2]:
!pip install transformers torch seqeval accelerate

In [3]:
import torch
from transformers import Trainer

In [4]:
import datasets
from datasets import load_dataset, concatenate_datasets
import pandas as pd
from transformers import AutoModelForTokenClassification, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import torch
from transformers import DataCollatorForTokenClassification
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import TrainingArguments, Trainer
import numpy as np

In [5]:
naamapadam_hi = load_dataset("ai4bharat/naamapadam", 'hi', split="train[:10000]")  # Take only 10000 samples
naamapadam_ml = load_dataset("ai4bharat/naamapadam", 'ml', split="train[:10000]")  # Take only 10000 samples
naamapadam = concatenate_datasets([naamapadam_hi, naamapadam_ml])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


0000.parquet:   0%|          | 0.00/93.3M [00:00<?, ?B/s]

0001.parquet:   0%|          | 0.00/5.48M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/93.5k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

...

0000.parquet:   0%|          | 0.00/81.5k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/315k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/716652 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/974 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3618 [00:00<?, ? examples/s]

In [ ]:
#not using this now, we just need a proof of concept
naamapadam_hi = load_dataset("ai4bharat/naamapadam", 'hi')
naamapadam_ml = load_dataset("ai4bharat/naamapadam", 'ml')
naamapadam = concatenate_datasets([naamapadam_hi['train'], naamapadam_ml['train']])

In [6]:
model_name = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(naamapadam.features['ner_tags'].feature.names))

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def preprocess_function(examples):
    tokens = examples['tokens']
    ner_tags = examples['ner_tags']

    tokenized_inputs = tokenizer(
        tokens,
        truncation=True,
        padding=True,
        max_length=128,
        is_split_into_words=True,
        return_offsets_mapping=True
    )

    labels = []
    for i, label in enumerate(ner_tags):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [8]:
tokenized_datasets = naamapadam.map(preprocess_function, batched=True)
train_testvalid_datasets = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_testvalid_datasets['train']
testvalid_dataset =  train_testvalid_datasets['test'].train_test_split(test_size=0.5)
test_dataset = testvalid_dataset['train']
valid_dataset = testvalid_dataset['test']

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [9]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [10]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [naamapadam.features['ner_tags'].feature.names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [naamapadam.features['ner_tags'].feature.names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = {
        "accuracy": accuracy_score(true_labels, true_predictions),
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
    }
    return results

In [11]:
def save_model_with_contiguous_tensors(trainer: Trainer, output_dir: str):
    state_dict = trainer.model.state_dict()
    contiguous_state_dict = {k: v.contiguous() if isinstance(v, torch.Tensor) else v
                             for k, v in state_dict.items()}
    trainer.model.save_pretrained(output_dir, state_dict=contiguous_state_dict)
    trainer.tokenizer.save_pretrained(output_dir)

class ContiguousTrainer(Trainer):
    def save_model(self, output_dir=None, _internal_call=False):
        output_dir = output_dir if output_dir is not None else self.args.output_dir
        save_model_with_contiguous_tensors(self, output_dir)


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/BTP_yash",
    evaluation_strategy="steps",
    #eval_steps=50,
    logging_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
#not using this for now, will take too long
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/BTP_yash",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer = ContiguousTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,0.967400,0.732398,0.801061,0.000000,0.000000,0.000000
100,0.687000,0.639398,0.804425,0.123762,0.006555,0.012450
150,0.626600,0.581731,0.827815,0.481159,0.130572,0.205403
200,0.559900,0.559085,0.832505,0.424267,0.276875,0.335079
250,0.556200,0.516000,0.848922,0.526873,0.339276,0.412759
300,0.522200,0.495101,0.851853,0.573406,0.304143,0.397464
350,0.508700,0.474538,0.857099,0.576109,0.364185,0.446265
400,0.453100,0.460635,0.860092,0.572243,0.394599,0.467101
450,0.464700,0.452314,0.863579,0.534288,0.420818,0.470813
500,0.455000,0.432007,0.869287,0.616372,0.402727,0.487155


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=3000, training_loss=0.38317291164398193, metrics={'train_runtime': 2182.6509, 'train_samples_per_second': 21.992, 'train_steps_per_second': 1.374, 'total_flos': 265147379712000.0, 'train_loss': 0.38317291164398193, 'epoch': 3.0})

In [15]:
trainer.save_model("./ner_model_small")

In [16]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

{'eval_loss': 0.3494057357311249, 'eval_accuracy': 0.8923586544027786, 'eval_precision': 0.6, 'eval_recall': 0.5768595041322314, 'eval_f1': 0.5882022471910112, 'eval_runtime': 17.8394, 'eval_samples_per_second': 112.111, 'eval_steps_per_second': 7.007, 'epoch': 3.0}


In [ ]:
# Print the first 5 samples from the Hindi dataset
print("First 5 samples from Hindi (hi) dataset:")
for i in range(5):
    print(naamapadam_hi[i])

# Print the first 5 samples from the Malayalam dataset
print("\nFirst 5 samples from Malayalam (ml) dataset:")
for i in range(5):
    print(naamapadam_ml[i])

First 5 samples from Hindi (hi) dataset:
{'tokens': ['सेक्टर', '55/56', 'के', 'एसएचओ', 'अरविंद', 'कुमार', 'ने', 'बताया', 'कि', 'इस', 'मामले', 'में', 'आईपीसी', 'की', 'धारा', '376', '-', 'डी', '(', 'गैंगरेप', ')', 'के', 'तहत', 'मामला', 'दर्ज', 'कर', 'लिया', 'गया', 'है', '।'], 'ner_tags': [0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'tokens': ['मध्य', 'रेलवे', 'एवं', 'पश्चिम', 'रेलवे', 'के', 'अधिकारियों', 'के', 'अनुसार', 'मनमाड', '-', 'मुंबई', 'एक्सप्रेस', ',', 'गुजरात', 'एक्सप्रेस', ',', 'सौराष्ट्र', 'एक्सप्रेस', ',', 'बांद्रा', 'टर्मिनस', 'सूरत', 'इंटरसिटी', 'एक्सप्रेस', 'और', 'मुंबई', 'सेंट्रल', '-', 'अहमदाबाद', 'शताब्दी', 'एक्सप्रेस', 'जैसी', 'कुछ', 'इंटरसिटी', 'ट्रेनों', 'को', 'रद्द', 'कर', 'दिया', 'गया', 'है', '।'], 'ner_tags': [3, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 3, 4, 0, 3, 4, 0, 3, 4, 0, 5, 5, 0, 0, 0, 0, 3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'tokens': ['जाने', '-', 'माने', 'वैज्ञानिक', 'सिवान', 'के', '.', 'को', 'भारतीय'

In [19]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Load the trained model and tokenizer from the checkpoint
model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/BTP_yash/checkpoint-3000") #train the model and give your own path here
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/BTP_yash/checkpoint-3000")

# Create a NER pipeline for inference
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Test with a sample sentence in Hindi
sentence = "जाने-माने वैज्ञानिक सिवान को भारतीय अंतरिक्ष अनुसंधान संगठन (इसरो) का अध्यक्ष नियुक्त किया गया है।"  # Replace this with your own sentence for testing
sentence2 = "ടോവിനോ തോമസ് നായകനായെത്തുന്ന പുതിയ ചിത്രമായ കൽക്കിയുടെ ചിത്രീകരണം ആരംഭിച്ചു." #similarly as above

# Get the NER predictions
ner_results = ner_pipeline(sentence2)

# Print the results
for entity in ner_results:
    print(f"Entity: {entity['word']}, Label: {entity['entity']}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Entity: ▁ട, Label: LABEL_1
Entity: വന, Label: LABEL_2
Entity: ▁തമ, Label: LABEL_2
Entity: സ, Label: LABEL_2
Entity: ▁നയ, Label: LABEL_0
Entity: ക, Label: LABEL_0
Entity: നയ, Label: LABEL_0
Entity: ത, Label: LABEL_0
Entity: തന, Label: LABEL_0
Entity: ന, Label: LABEL_0
Entity: ▁പത, Label: LABEL_0
Entity: യ, Label: LABEL_0
Entity: ▁ച, Label: LABEL_0
Entity: തര, Label: LABEL_0
Entity: മയ, Label: LABEL_0
Entity: ▁കൽ, Label: LABEL_5
Entity: ക, Label: LABEL_0
Entity: ക, Label: LABEL_0
Entity: യട, Label: LABEL_0
Entity: ▁ച, Label: LABEL_0
Entity: തര, Label: LABEL_0
Entity: കരണ, Label: LABEL_0
Entity: ▁ആ, Label: LABEL_0
Entity: രഭ, Label: LABEL_0
Entity: ച, Label: LABEL_0
Entity: ച, Label: LABEL_0
Entity: ., Label: LABEL_0


sample_data
